In [1]:
import pypsa
import pandas as pd

In [2]:
n = pypsa.Network("networks/Sample/2018_181_90m_c1.25_Co2L0.0-1H.nc")

INFO:pypsa.io:Imported network 2018_181_90m_c1.25_Co2L0.0-1H.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units


In [10]:
# Assume 'snapshot_to_exclude' is the snapshot you don't want to include in the solve
snapshot_to_exclude = '2024-10-10 12:00'  # Example timestamp, change it accordingly

# Filter the snapshots to exclude the specific one
snapshots_to_solve = n.snapshots[n.snapshots != snapshot_to_exclude]

# Now solve the n with the remaining snapshots
#n.lopf(snapshots_to_solve, solver_name='gurobi')

In [3]:
# Assume 'snapshot_to_remove' is the snapshot you want to remove
snapshot_to_remove = '2019-10-10 12:00'  # Example timestamp, change it accordingly

# Remove the snapshot from the n's snapshots
n.snapshots = n.snapshots.drop(snapshot_to_remove)

# Now solve the n without the removed snapshot
#n.lopf(n.snapshots, solver_name='gurobi')

In [27]:
# Step 1: Define the start and end of the period to remove
start_period = "2019-01-20 15:00:00"
end_period = "2019-01-24 21:00:00"

# Step 2: Identify snapshots to exclude
snapshots_to_exclude = n.snapshots[(n.snapshots >= start_period) & (n.snapshots <= end_period)]

# Step 3: Remove the identified snapshots from the network using drop
n.snapshots = n.snapshots.drop(snapshots_to_exclude)

## Print remaining snapshots
#df = pd.DataFrame(n.snapshots[n.snapshots.month == 1])
#pd.set_option('display.max_rows', None)  # Show all rows
#print(df)
#pd.reset_option('display.max_rows')

In [9]:
import pypsa
import pandas as pd



n = pypsa.Network("networks/Sample/2018_181_90m_c1.25_Co2L0.0-1H.nc")

# Step 1: Read the CSV file with periods to exclude
exclude_periods = pd.read_csv('difficult_periods.csv', parse_dates=['start', 'end'])
# Step 2: Identify snapshots to exclude
snapshots_to_exclude = set()

for index, row in exclude_periods.iterrows():
    start_time = row['start'].strftime("%Y-%m-%d %H:%M:%S")  # Convert to string
    end_time = row['end'].strftime("%Y-%m-%d %H:%M:%S")      # Convert to string

    # Check if snapshots in the n fall within the start and end range
    snapshots_to_exclude.update(n.snapshots[(n.snapshots >= start_time) & (n.snapshots <= end_time)])
    
# Convert to list for filtering
snapshots_to_exclude = list(snapshots_to_exclude)

# Step 3: Remove excluded snapshots from the n
n.snapshots = n.snapshots.drop(snapshots_to_exclude)


n.export_to_netcdf("networks/outputs/unsolved/2018_ExcludeSnapshot.nc")

/tmp/ipykernel_4686/213071099.py:6: FutureWarning: Inferring datetime64[ns, UTC] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns, UTC])
  for index, row in exclude_periods.iterrows():


In [10]:
n.export_to_netcdf("networks/outputs/unsolved/2018_ExcludeSnapshot.nc")

INFO:pypsa.io:Exported network 2018_ExcludeSnapshot.nc has storage_units, links, generators, carriers, buses, lines, global_constraints, loads


<xarray.Dataset> Size: 384MB
Dimensions:                               (snapshots: 8657,
                                           investment_periods: 0,
                                           storage_units_i: 284,
                                           storage_units_t_inflow_i: 55,
                                           storage_units_t_p_i: 284,
                                           storage_units_t_p_dispatch_i: 284,
                                           ...
                                           lines_t_p1_i: 144,
                                           lines_t_mu_lower_i: 144,
                                           lines_t_mu_upper_i: 144,
                                           global_constraints_i: 2,
                                           loads_i: 90, loads_t_p_set_i: 90,
                                           loads_t_p_i: 90)
Coordinates: (12/35)
  * snapshots                             (snapshots) int64 69kB 0 1 ... 8656
  * investment_periods                    (investment_periods) float64 0B 
  * storage_units_i                       (storage_units_i) object 2kB 'AL0 0...
  * storage_units_t_inflow_i              (storage_units_t_inflow_i) object 440B ...
  * storage_units_t_p_i                   (storage_units_t_p_i) object 2kB 'A...
  * storage_units_t_p_dispatch_i          (storage_units_t_p_dispatch_i) object 2kB ...
    ...                                    ...
  * lines_t_mu_lower_i                    (lines_t_mu_lower_i) object 1kB '1'...
  * lines_t_mu_upper_i                    (lines_t_mu_upper_i) object 1kB '1'...
  * global_constraints_i                  (global_constraints_i) object 16B '...
  * loads_i                               (loads_i) object 720B 'AL0 0' ... '...
  * loads_t_p_set_i                       (loads_t_p_set_i) object 720B 'AL0 ...
  * loads_t_p_i                           (loads_t_p_i) object 720B 'AL0 0' ....
Data variables: (12/101)
    snapshots_snapshot                    (snapshots) datetime64[ns] 69kB 201...
    snapshots_objective                   (snapshots) float64 69kB 1.0 ... 1.0
    snapshots_generators                  (snapshots) float64 69kB 1.0 ... 1.0
    snapshots_stores                      (snapshots) float64 69kB 1.0 ... 1.0
    investment_periods_objective          (investment_periods) float64 0B 
    investment_periods_years              (investment_periods) float64 0B 
    ...                                    ...
    global_constraints_carrier_attribute  (global_constraints_i) object 16B '...
    global_constraints_sense              (global_constraints_i) object 16B '...
    global_constraints_constant           (global_constraints_i) float64 16B ...
    loads_bus                             (loads_i) object 720B 'AL0 0' ... '...
    loads_t_p_set                         (snapshots, loads_t_p_set_i) float64 6MB ...
    loads_t_p                             (snapshots, loads_t_p_i) float64 6MB ...
Attributes:
    network__linearized_uc:      0
    network__multi_invest:       0
    network_name:                
    network_objective:           130826202021.79271
    network_objective_constant:  5766704420.833668
    network_pypsa_version:       0.22.0
    network_srid:                4326
    meta:                        {"version": "0.7.0", "tutorial": false, "log...

## Exclude snapshots for each network and export netowrks

In [3]:
import pypsa
import pandas as pd
import subprocess
import glob
import os

# Step 1: Run "scripts/difficult_periods.py" to generate "difficult_periods.csv"
#subprocess.run(["python", "scripts/difficult_periods_MODIFIED.py"], check=True)
!NETWORK_FOLDER="networks/sample" python scripts/difficult_periods_MODIFIED.py

# Step 2: Read the CSV file with periods to exclude
exclude_periods = pd.read_csv('difficult_periods.csv', parse_dates=['start', 'end'])

# Step 3: Process each network file in the "networks/sample" folder
network_files = glob.glob("networks/sample/*.nc")

for network_file in network_files:
    print(f"Processing {network_file}...")

    # Load the network
    n = pypsa.Network(network_file)
    
    # Step 4: Identify snapshots to exclude
    snapshots_to_exclude = set()
    
    for _, row in exclude_periods.iterrows():
        start_time = row['start'].strftime("%Y-%m-%d %H:%M:%S")  # Convert to string
        end_time = row['end'].strftime("%Y-%m-%d %H:%M:%S")      # Convert to string

        # Check if snapshots in n fall within the start and end range
        snapshots_to_exclude.update(n.snapshots[(n.snapshots >= start_time) & (n.snapshots <= end_time)])
        
    # Convert to list for filtering
    snapshots_to_exclude = list(snapshots_to_exclude)

    # Get the remaining snapshots
    remaining_snapshots = n.snapshots.drop(snapshots_to_exclude)

    # Check if any snapshots were actually excluded
    if set(remaining_snapshots) == set(n.snapshots):
        print(f"No snapshot changes for {network_file}. Skipping optimization.")
        continue  # Skip to the next network file if no snapshots were excluded

    # Step 5: Remove excluded snapshots from n
    n.snapshots = n.snapshots.drop(snapshots_to_exclude)

    # Step 6: Export modified network
    output_filename = os.path.join("networks/outputs/unsolved", os.path.basename(network_file).replace(".nc", "_SC.nc"))
    n.export_to_netcdf(output_filename)
    print(f"Saved modified network to {output_filename}")

/home/cs/miniforge3/envs/RD_env/lib/python3.12/pty.py:95: DeprecationWarning: This process (pid=49426) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


INFO:pypsa.io:Imported network 2019_181_90m_c1.25_Co2L0.0-1H.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units
INFO:pypsa.io:Imported network 2018_181_90m_c1.25_Co2L0.0-1H.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units
INFO:pypsa.io:Imported network 2017_181_90m_c1.25_Co2L0.0-1H.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units
INFO:root:Identified 1 difficult periods.
Processing networks/sample/2019_181_90m_c1.25_Co2L0.0-1H.nc...


INFO:pypsa.io:Imported network 2019_181_90m_c1.25_Co2L0.0-1H.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units


No snapshot changes for networks/sample/2019_181_90m_c1.25_Co2L0.0-1H.nc. Skipping optimization.
Processing networks/sample/2018_181_90m_c1.25_Co2L0.0-1H.nc...


INFO:pypsa.io:Imported network 2018_181_90m_c1.25_Co2L0.0-1H.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units
INFO:pypsa.io:Exported network '2018_181_90m_c1.25_Co2L0.0-1H_SC.nc' contains: buses, lines, links, carriers, loads, global_constraints, generators, storage_units


Saved modified network to networks/outputs/unsolved/2018_181_90m_c1.25_Co2L0.0-1H_SC.nc
Processing networks/sample/2017_181_90m_c1.25_Co2L0.0-1H.nc...


INFO:pypsa.io:Imported network 2017_181_90m_c1.25_Co2L0.0-1H.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units


No snapshot changes for networks/sample/2017_181_90m_c1.25_Co2L0.0-1H.nc. Skipping optimization.


## Exclude snapshots and solve directly

In [ ]:
import pypsa
import pandas as pd
import glob
import os
import subprocess

# Step 0: Run "scripts/difficult_periods.py" to generate "difficult_periods.csv"
subprocess.run(["python", "scripts/difficult_periods_MODIFIED.py"], check=True)

# Step 1: Read the CSV file with periods to exclude
exclude_periods = pd.read_csv('difficult_periods.csv', parse_dates=['start', 'end'])

# Step 2: Set up output directory for solved networks
output_directory = "networks/outputs/solved"
os.makedirs(output_directory, exist_ok=True)

# Step 3: Process each network file in the "networks/sample" folder
network_files = glob.glob("networks/sample/*.nc")

for network_file in network_files:
    print(f"Processing {network_file}...")

    # Load the network
    n = pypsa.Network(network_file)
    
    # Step 4: Identify snapshots to exclude
    snapshots_to_exclude = set()
    
    for _, row in exclude_periods.iterrows():
        start_time = row['start'].strftime("%Y-%m-%d %H:%M:%S")  # Convert to string
        end_time = row['end'].strftime("%Y-%m-%d %H:%M:%S")      # Convert to string

        # Check if snapshots in n fall within the start and end range
        snapshots_to_exclude.update(n.snapshots[(n.snapshots >= start_time) & (n.snapshots <= end_time)])
        
    # Convert to list for filtering
    snapshots_to_exclude = list(snapshots_to_exclude)

    # Get the remaining snapshots
    remaining_snapshots = n.snapshots.drop(snapshots_to_exclude)

    # Check if any snapshots were actually excluded
    if set(remaining_snapshots) == set(n.snapshots):
        print(f"No snapshot changes for {network_file}. Skipping optimization.")
        continue  # Skip to the next network file if no snapshots were excluded

    # Step 5: Solve only for the remaining snapshots
    n.optimize(solver_name='gurobi', snapshots=remaining_snapshots)

    # Step 6: Export modified network
    output_filename = os.path.join(output_directory, os.path.basename(network_file).replace(".nc", "_SC_solved.nc"))
    n.export_to_netcdf(output_filename)
    print(f"Solved and saved modified network to {output_filename}")
